# There's No Place Like Home

In [1]:
# The code was removed by Watson Studio for sharing.

!pip install folium

In [160]:
import json # library to handle JSON files
import requests # library to handle requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium
from geopy.geocoders import Nominatim
import math
import matplotlib.cm as cm
import matplotlib.colors as colors
import statistics

In [3]:
g_code0 = 'Kurfürstenallee 9 Bonn Germany'
g_code1 = 'London'

In [4]:
geolocator = Nominatim(user_agent="No_place_like_home")
g_loc = geolocator.geocode(g_code0)
print(g_loc.address)
print((g_loc.latitude, g_loc.longitude))

Musikschule der Stadt Bonn, 9, Kurfürstenallee, Alt-Godesberg, Bad Godesberg, Bonn, Nordrhein-Westfalen, 53177, Deutschland
(50.67920355, 7.155436486997861)


#### First we need to compose a portrait of present neighbourhood 

We need to know coordinates of every circle center and for this we well use a little bit of trigonometry to calculate how geographical coordinates correspond with distance in metric units at given latitude and longitude of the place. 

In [5]:
Earth_rad_la = 6356752.3 #Earth polar radius will be used to calculate latitude distances
Earth_rad_lo = 6378137 #Earth equatorial radius will be used to calculate longitude distances 

def Grad_step (dist, lat): # We need only latitude of the place and the distance
    
    grad_la = (180*dist)/(Earth_rad_lo*math.pi) 
    # Each meridian is equal in size, so we calculate angle just by dividing target distance to Earth polar radius 
        
    grad_lo = (180 * dist) / (Earth_rad_la * math.cos( lat*math.pi / 180) * math.pi) 
    # Here we calculate Earth radius at a givel latitude and then apply the same formula to calculate angle
    # but we also use equatorial radius as a base
    
    return (grad_la, grad_lo)

In [7]:
g_loc_m = [g_loc.latitude,g_loc.longitude]

tor_m = folium.Map(location=g_loc_m,zoom_start=15)

folium.Marker(g_loc_m).add_to(tor_m)

folium.Circle(
    #bounds=bounds_m,
    location = g_loc_m,
    radius=500,
    popup=g_loc.address,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(tor_m)

tor_m

In [8]:
radius = 500
LIMIT = 100
m_latitude = g_loc.latitude
m_longitude = g_loc.longitude

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, m_latitude, m_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=51W22WEGRWBLY2FOSYRLSBNR3BLM5WC1MYER00WPBOAWAZUX&client_secret=IJRAYGRRWPIRTY0SPIALYO1PGYHN2TQNG3ADVQI5TM1B3MDZ&ll=50.67920355,7.155436486997861&v=20180604&radius=500&limit=100'

In [9]:
# results = json_normalize(requests.get(url).json())
results = requests.get(url).json()["response"]['groups'][0]['items']

venues_list=[]

venues_list.append([(
#            name, 
#            lat, 
#            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [#'Neighborhood', 
                #'Neighborhood Latitude', 
                #'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category',
                'CategoryID']

nearby_venues.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,CategoryID
0,Terra Vino,50.680863,7.149654,Italian Restaurant,4bf58dd8d48988d110941735
1,Fischgeschäft Stuch,50.683327,7.153269,Fish Market,4bf58dd8d48988d10e951735
2,La Redoute,50.681829,7.152891,Event Space,4bf58dd8d48988d171941735
3,dm-drogerie markt,50.682312,7.151922,Drugstore,5745c2e4498e11e7bccabdbd
4,Ruby's Baguetterie,50.683362,7.156062,French Restaurant,4bf58dd8d48988d10c941735


In [10]:
nearby_venues.shape

(20, 5)

In [11]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 16 uniques categories.


In [12]:
# uniq_ven = pd.DataFrame(nearby_venues['Venue Category'].unique())
uniq_ven = nearby_venues['Venue Category'].unique()
uniq_ven

array(['Italian Restaurant', 'Fish Market', 'Event Space', 'Drugstore',
       'French Restaurant', 'Park', 'Clothing Store', 'Hotel', 'Theater',
       'Shopping Mall', 'Metro Station', 'Tunnel', 'Flower Shop',
       'Burger Joint', 'Video Store', 'Pizza Place'], dtype=object)

In [25]:
uniq_catid = nearby_venues['CategoryID'].unique()
uniq_catid

array(['4bf58dd8d48988d110941735', '4bf58dd8d48988d10e951735',
       '4bf58dd8d48988d171941735', '5745c2e4498e11e7bccabdbd',
       '4bf58dd8d48988d10c941735', '4bf58dd8d48988d163941735',
       '4bf58dd8d48988d103951735', '4bf58dd8d48988d1fa931735',
       '4bf58dd8d48988d137941735', '4bf58dd8d48988d1fd941735',
       '4bf58dd8d48988d1fd931735', '52f2ab2ebcbc57f1066b8b4a',
       '4bf58dd8d48988d11b951735', '4bf58dd8d48988d16c941735',
       '4bf58dd8d48988d126951735', '4bf58dd8d48988d1ca941735'],
      dtype=object)

In [13]:
n_ven = nearby_venues.groupby('Venue Category').count()
n_ven.reset_index(inplace=True)
n_ven.sort_values('Venue',ascending = False,inplace=True)
n_ven.reset_index(drop=True,inplace=True)
n_ven.head()

,Venue Category,Venue,Venue Latitude,Venue Longitude,CategoryID
0,Burger Joint,2,2,2,2
1,Clothing Store,2,2,2,2
2,Drugstore,2,2,2,2
3,Theater,2,2,2,2
4,Event Space,1,1,1,1


#### Let's look at our venues on the map

In [194]:
g_loc_m = [g_loc.latitude,g_loc.longitude]

tor_m = folium.Map(location=g_loc_m,zoom_start=15)

colors_array = cm.rainbow(np.linspace(0, 1, n_ven.shape[0]))
rainbow = [colors.rgb2hex(i) for i in colors_array]

folium.Circle(
        location = g_loc_m,
        radius=500,
        #popup=geo_rev(new_lat[ind]),
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(tor_m)

for lat,lon,nam,cat in zip(nearby_venues['Venue Latitude'],nearby_venues['Venue Longitude'],nearby_venues['Venue'],nearby_venues['Venue Category']):
    
    label = folium.Popup(str(cat) + ' ' + str(nam), parse_html=True)

#folium.Marker(g_loc_m00).add_to(tor_m00)
    col_mar = n_ven.index[n_ven['Venue Category'] == cat][0]
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[col_mar],
        fill=True,
        fill_color=rainbow[col_mar-1],
        fill_opacity=0.7
        ).add_to(tor_m)

tor_m

In [15]:
# one hot encoding
nearby_venues['Neighborhood'] = 'Home'

nearby_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nearby_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_onehot.columns[-1]] + list(nearby_onehot.columns[:-1])
nearby_onehot = nearby_onehot[fixed_columns]

print(nearby_onehot.shape)
nearby_onehot.head()

(20, 17)


,Neighborhood,Burger Joint,Clothing Store,Drugstore,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
0,Home,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Home,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Home,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Home,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Home,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [117]:
nearby_grouped = nearby_onehot.groupby('Neighborhood').mean().reset_index() 
# These will be used as weights in recomendation system
nearby_grouped.head()

,Neighborhood,Burger Joint,Clothing Store,Drugstore,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
0,Home,0.1,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.1,0.05,0.05


#### Now let's go to our destination city and find, what neighbourhood is the most similar to this one

In [17]:
geolocator = Nominatim(user_agent="No_place_like_home")
g_loc1 = geolocator.geocode(g_code1)
print(g_loc1.address)
print((g_loc1.latitude, g_loc1.longitude))

London, Greater London, England, SW1A 2DX, United Kingdom
(51.5073219, -0.1276474)


In [18]:
g_loc_m1 = [g_loc1.latitude,g_loc1.longitude]

tor_m1 = folium.Map(location=g_loc_m1,zoom_start=14)

folium.Marker(g_loc_m1).add_to(tor_m1)

tor_m1

#### Now we need to split our target city into squares (circles, actually) of fixed radius evenly covering the map. 

In [19]:
step_la, step_lo = Grad_step (500, g_loc1.latitude)
print (step_la, step_lo)

0.004491576420597607 0.007240650114328457


#### II want my circles to be placed so as to cover all the map while minimally overlapping 

In [28]:
circ_rad = 7

# Number of steps we take from the center of the city in each direction
# The final number of circles by each side will be 2*circ_rad + 1 

overlap = 2/math.sqrt(2)

s_la = step_la * overlap
s_lo = step_lo * overlap

squares_lan = np.arange(g_loc1.latitude + s_la*(-circ_rad), g_loc1.latitude + s_la*(circ_rad+1), s_la)
squares_lon = np.arange(g_loc1.longitude + s_lo*(-circ_rad), g_loc1.longitude + s_lo*(circ_rad+1), s_lo)

squares_la = []
squares_lo = []

for i in range(circ_rad*2+1): 
    for j in range(circ_rad*2+1):
        squares_la.append(squares_lan[i])
        squares_lo.append(squares_lon[j])

In [29]:
g_loc_m01 = [g_loc1.latitude,g_loc1.longitude]

tor_m01 = folium.Map(location=g_loc_m01,zoom_start=12)

folium.Marker(g_loc_m01).add_to(tor_m01)

for sq in range(len(squares_la)):
    folium.Circle(
        location=[squares_la[sq],squares_lo[sq]],
        radius=500,
        popup=g_loc1.address,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(tor_m01)

tor_m01

#### Here we see that circles are placed just as planned. 

There're 15x15=225 circles in total covering area of approx. 100 sq.km. We can assume that everythig interesting is included in this area)

Let's examine what's in each of these.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            u_c)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'CategoryID']
    
    return(nearby_venues)

In [30]:
n_names = int(math.pow(circ_rad*2+1,2))
print (n_names)
s_names = np.array(['Region #'+ str(reg) for reg in range(n_names)])
# s_names

225


In [31]:
lat = g_loc1.latitude
lng = g_loc1.longitude

u_c = ""
for ind in range(len(uniq_catid)-1):
    u_c += str(uniq_catid[ind])
    u_c += ","
u_c += str(uniq_catid[ind])

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            u_c)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=51W22WEGRWBLY2FOSYRLSBNR3BLM5WC1MYER00WPBOAWAZUX&client_secret=IJRAYGRRWPIRTY0SPIALYO1PGYHN2TQNG3ADVQI5TM1B3MDZ&v=20180604&ll=51.5073219,-0.1276474&radius=500&limit=100&categoryId=4bf58dd8d48988d110941735,4bf58dd8d48988d10e951735,4bf58dd8d48988d171941735,5745c2e4498e11e7bccabdbd,4bf58dd8d48988d10c941735,4bf58dd8d48988d163941735,4bf58dd8d48988d103951735,4bf58dd8d48988d1fa931735,4bf58dd8d48988d137941735,4bf58dd8d48988d1fd941735,4bf58dd8d48988d1fd931735,52f2ab2ebcbc57f1066b8b4a,4bf58dd8d48988d11b951735,4bf58dd8d48988d16c941735,4bf58dd8d48988d126951735,4bf58dd8d48988d126951735


#### Query is composed in such a way that only those categories present in our departure neighbourhood will be retrieved

In [32]:
u_c = ""
for ind in range(len(uniq_catid)-1):
    u_c += str(uniq_catid[ind])
    u_c += ","
u_c += str(uniq_catid[ind])

squares_venues = getNearbyVenues(names=s_names,
                                   latitudes=squares_la,
                                   longitudes=squares_lo
                                  )

Region #0
Region #1
Region #2
Region #3
Region #4
Region #5
Region #6
Region #7
Region #8
Region #9
Region #10
Region #11
Region #12
Region #13
Region #14
Region #15
Region #16
Region #17
Region #18
Region #19
Region #20
Region #21
Region #22
Region #23
Region #24
Region #25
Region #26
Region #27
Region #28
Region #29
Region #30
Region #31
Region #32
Region #33
Region #34
Region #35
Region #36
Region #37
Region #38
Region #39
Region #40
Region #41
Region #42
Region #43
Region #44
Region #45
Region #46
Region #47
Region #48
Region #49
Region #50
Region #51
Region #52
Region #53
Region #54
Region #55
Region #56
Region #57
Region #58
Region #59
Region #60
Region #61
Region #62
Region #63
Region #64
Region #65
Region #66
Region #67
Region #68
Region #69
Region #70
Region #71
Region #72
Region #73
Region #74
Region #75
Region #76
Region #77
Region #78
Region #79
Region #80
Region #81
Region #82
Region #83
Region #84
Region #85
Region #86
Region #87
Region #88
Region #89
Region #90
Region #9

In [38]:
print(squares_venues.shape)
squares_venues.head()

(4922, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,CategoryID
0,Region #0,51.462858,-0.199326,Wandsworth Park,51.462323,-0.204062,Park,4bf58dd8d48988d163941735
1,Region #0,51.462858,-0.199326,Marco Polo,51.462535,-0.199214,Italian Restaurant,4bf58dd8d48988d110941735
2,Region #0,51.462858,-0.199326,Third Door (Workhub & Nursery),51.461133,-0.199352,Coworking Space,4bf58dd8d48988d174941735
3,Region #0,51.462858,-0.199326,Bassline Productions,51.461341,-0.196844,Accessories Store,4bf58dd8d48988d102951735
4,Region #1,51.462858,-0.189086,Holiday Inn Express London - Wandsworth,51.462140,-0.188908,Hotel,4bf58dd8d48988d1fa931735


In [34]:
squares_venues.groupby('Neighborhood').count().sort_values('Venue Category', ascending = False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,CategoryID
Neighborhood,,,,,,,
Region #99,50,50,50,50,50,50,50
Region #144,50,50,50,50,50,50,50
Region #130,50,50,50,50,50,50,50
Region #131,50,50,50,50,50,50,50
Region #132,50,50,50,50,50,50,50
...,...,...,...,...,...,...,...
Region #58,2,2,2,2,2,2,2
Region #36,2,2,2,2,2,2,2
Region #24,2,2,2,2,2,2,2


####Let's save the result to avoid wasting time and resources making so many queries every time

In [35]:
squares_venues.to_csv('circles_london.csv', index=False)

Here we cut off all venues types not existing in our original neighbourhood. 
If they are present in a new neighbourhood, it's a bonus for our traveler but not a requirement.

In [43]:
sq_ven = squares_venues.loc[squares_venues['Venue Category'].isin(uniq_ven)]
print(sq_ven.shape)
sq_ven.head()

(4083, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,CategoryID
0,Region #0,51.462858,-0.199326,Wandsworth Park,51.462323,-0.204062,Park,4bf58dd8d48988d163941735
1,Region #0,51.462858,-0.199326,Marco Polo,51.462535,-0.199214,Italian Restaurant,4bf58dd8d48988d110941735
4,Region #1,51.462858,-0.189086,Holiday Inn Express London - Wandsworth,51.462140,-0.188908,Hotel,4bf58dd8d48988d1fa931735
5,Region #1,51.462858,-0.189086,The Alma Hotel,51.460867,-0.185335,Hotel,4bf58dd8d48988d1fa931735
6,Region #1,51.462858,-0.189086,Bramford Gardens,51.461597,-0.186918,Park,4bf58dd8d48988d163941735


In [44]:
sq_ven.groupby('Neighborhood').count().sort_values('Venue Category', ascending = False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,CategoryID
Neighborhood,,,,,,,
Region #123,50,50,50,50,50,50,50
Region #141,50,50,50,50,50,50,50
Region #76,50,50,50,50,50,50,50
Region #122,49,49,49,49,49,49,49
Region #121,49,49,49,49,49,49,49
...,...,...,...,...,...,...,...
Region #57,2,2,2,2,2,2,2
Region #25,1,1,1,1,1,1,1
Region #38,1,1,1,1,1,1,1


In [45]:
print('There are {} uniques categories.'.format(len(sq_ven['Venue Category'].unique())))

There are 15 uniques categories.


In [129]:
# one hot encoding
squares_onehot = pd.get_dummies(sq_ven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
squares_onehot['Neighborhood'] = sq_ven['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [squares_onehot.columns[-1]] + list(squares_onehot.columns[:-1])
squares_onehot = squares_onehot[fixed_columns]

print(squares_onehot.shape)
squares_onehot.head()

(4083, 16)


,Neighborhood,Burger Joint,Clothing Store,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
0,Region #0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Region #0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Region #1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,Region #1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,Region #1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [131]:
squares_grouped = squares_onehot.groupby('Neighborhood').sum().reset_index()
squares_grouped.iloc[:,1:].head()

,Burger Joint,Clothing Store,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
3,2,0,4,0,1,0,5,9,1,5,1,0,3,0,0
4,2,2,1,0,0,1,9,7,2,4,1,2,1,0,0


In [143]:
squares_grouped = squares_onehot.groupby('Neighborhood').sum().reset_index()

squares_grouped.iloc[:,1:] = squares_grouped.iloc[:,1:].applymap(lambda x: math.log(x+1))#,result_type='broadcast')

squares_grouped.head()

,Neighborhood,Burger Joint,Clothing Store,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
0,Region #0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.0
1,Region #1,0.000000,0.000000,0.693147,0.0,0.000000,0.000000,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.0
2,Region #10,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.000000,0.0,0.0
3,Region #100,1.098612,0.000000,1.609438,0.0,0.693147,0.000000,1.791759,2.302585,0.693147,1.791759,0.693147,0.000000,1.386294,0.0,0.0
4,Region #101,1.098612,1.098612,0.693147,0.0,0.000000,0.693147,2.302585,2.079442,1.098612,1.609438,0.693147,1.098612,0.693147,0.0,0.0


#### We will use content-based filtering to determine the best place to move 

In [144]:
nearby_p = nearby_grouped.T
squares_p = squares_grouped.T

pears = nearby_p.join(squares_p, lsuffix='index', rsuffix='index').T.set_index('Neighborhood').fillna(0)
pears.head(25)

,Burger Joint,Clothing Store,Drugstore,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
Neighborhood,,,,,,,,,,,,,,,,
Home,0.100000,0.100000,0.1,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.100000,0.050000,0.050000
Region #0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000
Region #1,0.000000,0.000000,0.0,0.693147,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000
Region #10,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000
Region #100,1.098612,0.000000,0.0,1.609438,0.000000,0.693147,0.000000,1.791759,2.302585,0.693147,1.791759,0.693147,0.000000,1.386294,0.000000,0.000000
Region #101,1.098612,1.098612,0.0,0.693147,0.000000,0.000000,0.693147,2.302585,2.079442,1.098612,1.609438,0.693147,1.098612,0.693147,0.000000,0.000000
Region #102,0.000000,0.693147,0.0,1.098612,0.000000,0.000000,1.386294,2.079442,2.197225,0.000000,1.791759,0.693147,0.000000,1.098612,0.000000,0.000000
Region #103,0.693147,0.693147,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,1.791759,0.000000,0.000000,0.000000,0.000000,0.000000
Region #104,0.693147,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.945910,0.000000,0.000000,0.693147,0.693147,0.000000


In [145]:
pears1 = pears.loc['Home']
pears.drop('Home',axis=0, inplace=True)
pears.head()  

,Burger Joint,Clothing Store,Drugstore,Event Space,Fish Market,Flower Shop,French Restaurant,Hotel,Italian Restaurant,Metro Station,Park,Pizza Place,Shopping Mall,Theater,Tunnel,Video Store
Neighborhood,,,,,,,,,,,,,,,,
Region #0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.0
Region #1,0.000000,0.000000,0.0,0.693147,0.0,0.000000,0.000000,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.0
Region #10,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.000000,0.0,0.0
Region #100,1.098612,0.000000,0.0,1.609438,0.0,0.693147,0.000000,1.791759,2.302585,0.693147,1.791759,0.693147,0.000000,1.386294,0.0,0.0
Region #101,1.098612,1.098612,0.0,0.693147,0.0,0.000000,0.693147,2.302585,2.079442,1.098612,1.609438,0.693147,1.098612,0.693147,0.0,0.0


p_corr = pears.corrwith(pears1, axis=1 ,method='pearson').sort_values(ascending=False) 
p_corr.index[0]
print('The most suitable place is ', p_corr.index[0], ' with correlation ', p_corr[0])
p_corr

In [146]:
recom_tab = ((pears * pears1).sum(axis=1)/pears1.sum()).sort_values(ascending=False)
recom_tab.head()

Neighborhood
Region #174    1.124896
Region #113    1.101304
Region #128    1.009600
Region #130    1.009000
Region #99     0.996238
dtype: float64

#### We may have more than 1 winner (which actually happened in this example), so we need to be prepared

In [147]:
num_win = 1
while recom_tab[num_win]==recom_tab[num_win-1]: num_win+=1
num_win

1

#### Let's show our new place on the map

In [148]:
n_cor = sq_ven[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']].groupby('Neighborhood').mean()

n_cor.head()

,Neighborhood Latitude,Neighborhood Longitude
Neighborhood,,
Region #0,51.462858,-0.199326
Region #1,51.462858,-0.189086
Region #10,51.462858,-0.096928
Region #100,51.500970,-0.096928
Region #101,51.500970,-0.086688


#### We will use reverse geolocator to determine address of a center of winner region based on it's coordinates

In [149]:
def geo_rev(lat):
    geolocator = Nominatim(user_agent="No_place_like_home")
    adr2 = str(lat[0]) + ',' + str(lat[1])
    location = geolocator.reverse(adr2)
    print(location.address,'\n')

In [150]:
new_lat = []
for ind in range(num_win):
    new_lat.append(n_cor.loc[recom_tab.index[ind],:])
    print(new_lat[ind],'\n')
    geo_rev(new_lat[ind])

Neighborhood Latitude     51.532730
Neighborhood Longitude    -0.107168
Name: Region #174, dtype: float64 

Kaplan, 10-14, White Lion Street, Angel, Pentonville, London Borough of Islington, London, Greater London, England, N1 9PF, United Kingdom 



In [151]:
new_lat[0][0]
g_loc_m2 = [new_lat[ind][0],new_lat[ind][1]]
g_loc_m2

[51.53273009316176, -0.10716774881583677]

#### And now let's look at the map to find the region where we should look for a new home 
We will also put all the venues on the map

In [193]:
if num_win ==1:
    g_loc_m02 =[new_lat[0][0],new_lat[0][1]]
else:
    g_loc_m02 =[statistics.mean(new_lat[:][0]),statistics.mean(new_lat[:][1])]

tor_m2 = folium.Map(location=g_loc_m02,zoom_start=14)

colors_array = cm.rainbow(np.linspace(0, 1, n_ven.shape[0]))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for ind in range(num_win):
    g_loc_m2 = [new_lat[ind][0],new_lat[ind][1]]
#    folium.Marker(g_loc_m2).add_to(tor_m2)

    folium.Circle(
        location = g_loc_m2,
        radius=500,
        popup=geo_rev(new_lat[ind]),
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(tor_m2)
    
    regio = sq_ven.loc[sq_ven['Neighborhood']== recom_tab.index[ind]]
    
    for lat,lon,nam,cat in zip(regio['Venue Latitude'],regio['Venue Longitude'],regio['Venue'],regio['Venue Category']):
    
        label = folium.Popup(str(cat) + ' ' + str(nam), parse_html=True)

        col_mar = n_ven.index[n_ven['Venue Category'] == cat][0]
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[col_mar],
            fill=True,
            fill_color=rainbow[col_mar-1],
            fill_opacity=0.7
            ).add_to(tor_m2)


tor_m2

Kaplan, 10-14, White Lion Street, Angel, Pentonville, London Borough of Islington, London, Greater London, England, N1 9PF, United Kingdom 



#### Let's compare old and new neighbourhoods head to head and look to what degree they are similar to each other in the end

In [154]:
def uni_cat (table):
    print(table.shape)
    i = 0

    while i < table.shape[0]:
        
        i += 1
        j = i-1
        
        while i < table.shape[0] and table.loc[i,'Venue Category'] == table.loc[j,'Venue Category']:
            table.loc[j,'Venue'] += (', ' + table.loc[i,'Venue'])
            table.loc[i,'Venue'] = 'To delete'
            i += 1
    
    table = table[table['Venue'] != 'To delete']
    table.index = pd.Series(range(table.shape[0]))
    
    print(table.shape)

    return(table)

In [155]:
sq_ven.head()
region=[]
reg1 = []
for ind in range(num_win):
    region.append(sq_ven.loc[sq_ven['Neighborhood']== recom_tab.index[0]])
    reg1.append(region[ind][['Venue','Venue Category']].sort_values('Venue Category').reset_index())
    reg1[ind].drop('index',axis=1,inplace=True)
    reg1[ind] = uni_cat(reg1[ind])
    reg1[ind].set_index('Venue Category',inplace=True)
    reg1[ind].rename(columns={"Venue": "New Venue"},inplace=True)
    #print(region[ind].shape)

#region.head()
# sq_ven[p_corr.index[0],'Venue Category'].unique()

(41, 2)
(14, 2)


In [156]:
reg0 = nearby_venues[['Venue','Venue Category']].sort_values('Venue Category').reset_index()#.set_index('Venue Category')

reg0.drop('index',axis=1,inplace=True)
reg0 = uni_cat(reg0)
reg0.set_index('Venue Category',inplace=True)
reg0.head()

(20, 2)
(16, 2)


,Venue
Venue Category,
Burger Joint,"Rosa's Burger, Beef Buddies"
Clothing Store,"H&M, C&A"
Drugstore,"dm-drogerie markt, Rossmann"
Event Space,La Redoute
Fish Market,Fischgeschäft Stuch


In [157]:
frames =[reg0]
for ind in range(num_win):
    reg1[ind] = reg1[ind].reindex(reg0.index)
    frames.append(reg1[ind])

pd.concat(frames, axis=1)

,Venue,New Venue
Venue Category,,
Burger Joint,"Rosa's Burger, Beef Buddies","Five Guys, Chapel Market Burger Stall (The Nak..."
Clothing Store,"H&M, C&A","Monsoon, Next, French Connection, H&M, Joy, GA..."
Drugstore,"dm-drogerie markt, Rossmann",NaN
Event Space,La Redoute,Lift
Fish Market,Fischgeschäft Stuch,Moxon's Fish Monger
Flower Shop,Blumen aus Holland,Angel Flowers
French Restaurant,Ruby's Baguetterie,"Le Coq Epicer, Frederick's Bar & Restaurant, C..."
Hotel,Insel Hotel Bonn,"DoubleTree by Hilton, Premier Inn London Angel..."
Italian Restaurant,Terra Vino,"Don Matteos, Dolcetto, Bella Italia, La Forche..."


#### Well, there's obviously no place like home. But we've found the closest replacement possible.